In [1]:
import numpy as np

# Analytical Solution for the Kinetic part

## Single particle

In [2]:
#spinnless single particle kinetic term
N = 4
n_avg = 0.5
t = -1.0
K = [[0 for i in range(0,N)] for j in range(0,N)]
for i in range(0,N):
    K[i][np.mod(i+1,N)] = t
    K[np.mod(i+1,N)][i] = t

In [3]:
e,y = np.linalg.eig(K)
psi = np.transpose(y)
order = np.argsort(e)
np.sort(e)

array([-2.00000000e+00,  0.00000000e+00,  6.59737022e-17,  2.00000000e+00])

In [4]:
np.amax(np.abs(np.dot(K,psi[order[0]] ) - e[order[0]]*psi[order[0]]))

8.881784197001252e-16

In [5]:
#spinless single particle kinetic energy levels
def E(n):
    return 2*t*np.cos(2*n*np.pi/N)

for n in range(N):
    print(E(n))

-2.0
-1.2246467991473532e-16
2.0
3.6739403974420594e-16


In [6]:
def psi_t(n):
    return np.array([np.exp(1j*2*n*i*np.pi/N) for i in range(N)])

n=3
np.amax(np.abs(np.dot(K,psi_t(n)) - E(n)*psi_t(n)))

7.347880794884119e-16

## Full Many Body Kinetic Term

In [7]:
import qiskit.quantum_info as qi

def X_label(i,Q):
    out = ''
    for j in range(0,Q):
        if i < Q-1:
            if i == j or i + 1 == j:
                out = out + 'X'
            else:
                out = out + 'I'
        else:
            if j == 0 or j == Q-1:
                out = out + 'X'
            else:
                out = out + 'Z'
    return out[::-1]

def Y_label(i,Q):
    out = ''
    for j in range(0,Q):
        if i < Q-1:
            if i == j or i + 1 == j:
                out = out + 'Y'
            else:
                out = out + 'I'
        else:
            if j == 0 or j == Q-1:
                out = out + 'Y'
            else:
                out = out + 'Z'
    return out[::-1]

def I_label(Q):
    out = ''
    for j in range(Q):
        out = out + 'I'
    return out


K_mb = 0*qi.Operator.from_label(I_label(N))
for i in range(0,N):
    K_mb = K_mb + 1/2*t*qi.Operator.from_label( X_label(i,N) )
    K_mb = K_mb + 1/2*t*qi.Operator.from_label( Y_label(i,N) )

    
    

In [8]:
e_mb,y_mb = np.linalg.eig(K_mb)
psi_mb = np.transpose(y_mb)
order_mb = np.argsort(e_mb)
#np.sort(e_mb)

In [9]:
# A function to print out the binary number
def bi(num,S):
    bi = bin(num)
    out = []
    Sdiff = S - len(bi) + 2
    for i in range(0,Sdiff):
        out.append(0)
    for i in range(2,len(bi)):
        out.append(int(bi[i]))
    return out

#Build the single particle wavefunction in the many body basis
def psi_mb1(n):
    i = 0
    out = [0 for m in range(2**N)]
    for m in range(2**N):
        if sum(bi(m,N)) == 1:
            out[m] = psi_t(n)[i]
            i += 1
    return np.array(out)

In [10]:
#Check the single particle state
n=2
np.amax(np.abs(np.dot(K_mb,psi_mb1(n)) - E(n)*psi_mb1(n)))

4.898587196589413e-16

### Two particle

In [11]:
#two particle energies
e2 = []
for m in range(2**N):
    state = bi(m,N)
    if sum(state) == 2:
        #print(state,e)
        em = np.dot(state,e)
        e2.append(em)
        
def E2(n,nn):
    return E(n)+E(nn)

#Slatter determinent
def slatter2(n,nn,x,xx):
    s = [[psi_t(n)[x], psi_t(n)[xx]],[psi_t(nn)[x], psi_t(nn)[xx]]]
    return np.linalg.det(s)

#two particle wavefunctions
def psi_mb2(n,nn):
    out = [0 for m in range(2**N)]
    for m in range(2**N):
        if sum(bi(m,N)) == 2:
            xs = np.argwhere(np.array(bi(m,N)) == 1 )
            x1 = xs[0,0]
            x2 = xs[1,0]
            out[m] = slatter2(n,nn,x1,x2)
    norm = np.sqrt(np.dot(np.conjugate(out),out))
    return np.array(out)/norm

In [12]:
n1 = 1
n2 = 3
np.amax(np.abs(np.dot(K_mb,psi_mb2(n1,n2)) - E2(n1,n2)*psi_mb2(n1,n2)))

2.4492935982947064e-16

### Three particle

In [13]:
#Energies
def E3(n1,n2,n3):
    return E(n1)+E(n2)+E(n3)

#Slatter determinent
def slatter(nl,xl):
    s = [[psi_t(n)[x] for x in xl] for n in nl]
    return np.linalg.det(s)

#two particle wavefunctions
def psi_mb3(n1,n2,n3):
    out = [0 for m in range(2**N)]
    for m in range(2**N):
        if sum(bi(m,N)) == 3:
            xs = np.argwhere(np.array(bi(m,N)) == 1 )
            x1 = xs[0,0]
            x2 = xs[1,0]
            x3 = xs[2,0]
            out[m] = slatter([n1,n2,n3],[x1,x2,x3])
    norm = np.sqrt(np.dot(np.conjugate(out),out))
    return np.array(out)/norm

In [14]:
n1 = 0
n2 = 2
n3 = 3
np.amax(np.abs(np.dot(K_mb,psi_mb3(n1,n2,n3)) - E3(n1,n2,n3)*psi_mb3(n1,n2,n3)))

2.449293598294707e-16

### General number of particles

In [15]:


#Energies
def En(nl):
    return sum([E(n) for n in nl])

#Slatter determinent
def slatter(nl,xl):
    s = [[psi_t(n)[x] for x in xl] for n in nl]
    return np.linalg.det(s)

#two particle wavefunctions
def psi_mbn(nl):
    out = [0 for m in range(2**N)]
    for m in range(2**N):
        if sum(bi(m,N)) == len(nl):
            xs = np.argwhere(np.array(bi(m,N)) == 1 )
            xl = [x[0] for x in xs]
            out[m] = slatter(nl,xl)
    norm = np.sqrt(np.dot(np.conjugate(out),out))
    return np.array(out)/norm

In [16]:
nl = [0,1,3,4]
np.amax(np.abs(np.dot(K_mb,psi_mbn(nl)) - En(nl)*psi_mbn(nl)))

3.9999999999999996

# U in the eigenbasis of K

In [17]:
def bkt(y1,o,y2):
    return np.dot(np.conjugate(y1),np.dot(o,y2))

In [18]:
def Z_label(i,Q):
    out = ''
    for j in range(0,Q):
        if i == j:
            out = out + 'Z'
        else:
            out = out + 'I'
    return out[::-1]


## Two Particle

Let us write down the basis for the two particle case

In [19]:
# The single spin basis in terms of the full single spin many body states
y_basis = []
for n in range(0,N):
    for m in range(n+1,N):
        y_basis.append(psi_mb2(n,m))

#The single spin basis in terms of the reduced single spin many body states
y2 = []
for n in range(0,len(y_basis)):
    ym = []
    for m in range(0,2**N):
        if sum(bi(m,N)) == 2:        
            ym.append(y_basis[n][m])
    y2.append(ym)

#The reduced two spin basis
y2_x = np.kron(y2,y2)


### Real space basis

Let us start with the real space representation of $\hat{U}$ and then transform it using the eigenvectors found for $\hat{K}$.  I will also write the full Hamiltonain in real space so I can compare the eigenvalues with the full Hamiltonain in the eigenbisis of $\hat{K}$

In [20]:
#Full U in real space
U_mb_x_full = 0*qi.Operator.from_label(I_label(N)+I_label(N))
for i in range(0,N):
    U_mb_x_full = U_mb_x_full + 1/4*qi.Operator.from_label(I_label(N)+I_label(N))
    U_mb_x_full = U_mb_x_full - 1/4*qi.Operator.from_label( I_label(N) + Z_label(i,N) )
    U_mb_x_full = U_mb_x_full - 1/4*qi.Operator.from_label( Z_label(i,N) + I_label(N) )
    U_mb_x_full = U_mb_x_full + 1/4*qi.Operator.from_label( Z_label(i,N) + Z_label(i,N) )
    
#two particle wavefunctions in real space
states = []
for m in range(0,2**N):
    if sum(bi(m,N)) == 2:
        states.append(m)

#two particle spinful wavefunction in real space
states_full = []
for i in range(0,len(states)):
    for j in range(0,len(states)):
        states_full.append(states[i]*2**N+states[j])

        
#two particle spinful wavefunction in real space as vectors
def psi_real_full(n):
    y = [0 for i in range(4**N)]
    y[states_full[n]] = 1
    return y

# Reduced U in real space
U_mb_x = []
for n in range(len(states_full)):
    U_mb_x_row = []
    for nn in range(len(states_full)):
        y1 = psi_real_full(n)
        y2 = psi_real_full(nn)
        Unm = bkt(y1,U_mb_x_full,y2)
        U_mb_x_row.append(Unm)
    U_mb_x.append(U_mb_x_row)
        

#Full kinetic part in real space
I = qi.Operator.from_label(I_label(N))
K_mb_x_full = np.kron(K_mb.data,I) + np.kron(I,K_mb.data)

#Reduced kinetic part in real space
K_mb_x = []
for n in range(len(states_full)):
    K_mb_x_row = []
    for nn in range(len(states_full)):
        y1 = psi_real_full(n)
        y2 = psi_real_full(nn)
        Knm = bkt(y1,K_mb_x_full,y2)
        K_mb_x_row.append(Knm)
    K_mb_x.append(K_mb_x_row)

### Return to eigenbasis

Now we can transform the real space $\hat{U}$ into the eigenbasis.

In [39]:
# Reduced spinful U in the eigenbasis
U_mb_n = []
for i in range(0,36):
    U_mb_n_row = []
    for j in range(0,36):
        U_mb_n_row.append(bkt(y2_x[i],U_mb_x,y2_x[j]))
    U_mb_n.append(U_mb_n_row)
    

import pandas as pd    
pd.DataFrame(np.real(U_mb_n)).round(3)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,1.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,...,0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00
1,0.00,1.00,-0.00,-0.00,0.00,-0.00,0.25,-0.00,-0.00,-0.00,...,-0.25,-0.25,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00
2,-0.00,-0.00,1.00,0.00,-0.00,-0.00,-0.00,0.25,-0.00,-0.00,...,-0.00,0.00,-0.25,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00
3,0.00,-0.00,0.00,1.00,-0.00,0.00,-0.00,0.25,-0.00,-0.00,...,0.00,-0.00,-0.25,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00
4,-0.00,0.00,-0.00,-0.00,1.00,0.00,0.00,0.00,0.25,0.25,...,-0.00,-0.00,-0.00,-0.25,0.00,-0.00,-0.00,-0.00,-0.00,0.00
5,0.00,-0.00,-0.00,0.00,0.00,1.00,-0.00,0.00,0.00,0.00,...,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,-0.00,0.00
6,-0.00,0.25,-0.00,-0.00,0.00,-0.00,1.00,-0.00,0.00,-0.00,...,-0.00,0.00,0.00,-0.00,-0.00,-0.25,0.00,-0.00,-0.00,0.00
7,-0.00,-0.00,0.25,0.25,0.00,0.00,-0.00,1.00,-0.00,-0.00,...,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.25,-0.25,0.00,-0.00
8,0.00,-0.00,-0.00,-0.00,0.25,0.00,0.00,-0.00,1.00,-0.00,...,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,-0.25,0.00
9,-0.00,-0.00,-0.00,-0.00,0.25,0.00,-0.00,-0.00,-0.00,1.00,...,0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,-0.25,-0.00


In [23]:
#The spinful energy levels
E_basis = []
for n in range(0,N):
    for m in range(n+1,N):
        E_basis.append(E2(n,m))
        
#Reduced spinful K in eigenbasis
K_mb_n = []
for e1 in E_basis:
    for e2 in E_basis:
        K_mb_n.append(e1+e2)
K_mb_n = np.diag(K_mb_n)

#Reduced spinful Hamiltonian in eigenbasis
#H_mb_n = U_mb_comp + K_mb_n
H_mb_n = U_mb_n + K_mb_n

#pd.DataFrame(H_mb_n)
e_n,y_n = np.linalg.eig(H_mb_n)
np.sort(e_n)

array([-3.34084762e+00-7.08806362e-31j, -3.29295138e+00-3.40281157e-17j,
       -2.86387634e+00-1.92086165e-17j, -2.78526086e+00+3.08763781e-31j,
       -1.56155281e+00+2.20922881e-16j, -1.56155281e+00+1.24900090e-16j,
       -1.00000000e+00+9.02056207e-17j, -1.00000000e+00-8.14306806e-17j,
       -1.00000000e+00+4.55364912e-17j, -1.00000000e+00-4.09828421e-17j,
       -5.61552813e-01+7.63277976e-17j, -5.61552813e-01+1.37844448e-16j,
        1.08957031e-15+9.18819034e-17j,  5.07690516e-01+1.65122939e-16j,
        5.23028722e-01+8.56701891e-17j,  1.00000000e+00-1.66533454e-16j,
        1.00000000e+00-1.11167501e-16j,  1.00000000e+00+1.11022302e-16j,
        1.00000000e+00+1.52655666e-16j,  1.00000000e+00+1.35308431e-16j,
        1.00000000e+00+5.55111512e-17j,  1.47697128e+00-1.15147649e-30j,
        1.49230948e+00-3.71419321e-30j,  2.00000000e+00-4.54112264e-17j,
        2.56155281e+00+4.51068761e-16j,  2.56155281e+00+3.46375102e-25j,
        3.00000000e+00-5.55111512e-17j,  3.00000000

In [24]:
#Reduced spinful Hamiltonian in real space
H_mb_x = np.array(K_mb_x) + np.array(U_mb_x)
e_x,y_x = np.linalg.eig(H_mb_x)
np.sort(e_x)

array([-3.34084762e+00+0.j, -3.29295138e+00+0.j, -2.86387634e+00+0.j,
       -2.78526086e+00+0.j, -1.56155281e+00+0.j, -1.56155281e+00+0.j,
       -1.00000000e+00+0.j, -1.00000000e+00+0.j, -1.00000000e+00+0.j,
       -1.00000000e+00+0.j, -5.61552813e-01+0.j, -5.61552813e-01+0.j,
        1.14454632e-16+0.j,  5.07690516e-01+0.j,  5.23028722e-01+0.j,
        1.00000000e+00+0.j,  1.00000000e+00+0.j,  1.00000000e+00+0.j,
        1.00000000e+00+0.j,  1.00000000e+00+0.j,  1.00000000e+00+0.j,
        1.47697128e+00+0.j,  1.49230948e+00+0.j,  2.00000000e+00+0.j,
        2.56155281e+00+0.j,  2.56155281e+00+0.j,  3.00000000e+00+0.j,
        3.00000000e+00+0.j,  3.00000000e+00+0.j,  3.00000000e+00+0.j,
        3.56155281e+00+0.j,  3.56155281e+00+0.j,  4.78526086e+00+0.j,
        4.86387634e+00+0.j,  5.29295138e+00+0.j,  5.34084762e+00+0.j])

In [25]:
np.amax(np.abs(np.sort(e_x)-np.sort(e_n)))

1.4210854715202004e-14

## Decompose U

Now we can decompose U into blocks, pad the blocks, and write those blocks in terms of Pauli operators.

In [26]:
def complete(b11):
    size = 2**np.ceil(np.log2(len(b11)))
    for v in b11:
        while len(v) < size:
            v.append(0.0)
    while len(b11) < size:
        i = len(b11)
        vnew = [0.0 for i in range(0,8)]
        vnew[i] = 10
        b11.append(vnew)
    return b11

In [28]:
import math
from qiskit.opflow import (I, X, Y, Z)
from qiskit.opflow.primitive_ops import MatrixOp

N = 4
S = 2
bL = math.comb(4,2)

blocks = {}
for bi in range(bL):
    for bj in range(bi,bL):
        bij = [[U_mb_n[i + bL*bi][j + bL*bj] for j in range(0,bL)] for i in range(0,bL)]
        if np.amax(np.abs(bij)) > 10**(-5):
            bij = complete(bij)
            bij_pauli = MatrixOp(bij).to_pauli_op()
            blocks[str(bi) + ',' + str(bj)] = bij_pauli

In [29]:
blocks.keys()

dict_keys(['0,0', '0,1', '0,2', '0,3', '0,4', '1,1', '1,2', '1,3', '1,5', '2,2', '2,4', '2,5', '3,3', '3,4', '3,5', '4,4', '4,5', '5,5'])

It looks like every block is used which means we do not really gain anything from using blocks.

In [504]:
pwd

'/Users/stenger/Documents/Research/Hubbard_symmetries'